In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append("../../../")
import os
from pathlib import Path

import geopandas as gpd
import pandas as pd

from povertymapping.iso3 import get_region_name
from povertymapping.rollout_grids import get_region_filtered_bingtile_grids

/home/jace/workspace/unicef-ai4d-poverty-mapping/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Generate Roll-out Grids: Timor Leste

This notebook generates the rollout grid tiles over the country. The output file is used as an input for Step 3, where we will run the trained model over the set of grids.

The generated grids are set at 2.4km (zoom level 14), matching the grids used during model training. The grids are also filtered to only include populated areas based on Meta's High Resolution Settlement Layer (HRSL) data.

## Set country-specific parameters

In [ ]:
COUNTRY_CODE = "tl"
REGION = get_region_name(COUNTRY_CODE, code="alpha-2").lower()
ADMIN_LVL = "ADM2"
ZOOM_LEVEL = 14

## Generate Grids

In [5]:
admin_grids_gdf = get_region_filtered_bingtile_grids(
    REGION, admin_lvl=ADMIN_LVL, quadkey_lvl=ZOOM_LEVEL, use_cache=True
)

2023-04-03 08:26:45.652 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:264 - No cached grids file found. Generating grids file :/home/jace/.cache/geowrangler/quadkey_grids/timor-leste_14_ADM2_populated_admin_grids.geojson
2023-04-03 08:26:45.655 | DEBUG    | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:281 - Loading boundaries for region timor-leste and admin level ADM2
2023-04-03 08:26:45.915 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:287 - Generating grids for region timor-leste and admin level ADM2 at quadkey level 14


2023-04-03 08:26:47.769 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:293 - Generated 2811 grids for region timor-leste and admin level ADM2 at quadkey level 14
2023-04-03 08:26:47.770 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:299 - Assigning grids to admin areas using metric crs epsg:3857
2023-04-03 08:26:48.504 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:305 - Getting timor-leste population data for filtering grids
2023-04-03 08:26:49.725 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:307 - Computing population zonal stats per grid
2023-04-03 08:26:49.726 | INFO     | povertymapping.rollout_grids:compute_raster_stats:98 - Creating raster zonal stats for 2811 grids for file size 829497978
2023-04-03 08:27:05.671 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:318 - Filtering unpopulated grids based on population data
2023-04-03 0

## Explore Populated Grids

In [6]:
admin_grids_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2024 entries, 0 to 2810
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geometry    2024 non-null   geometry
 1   quadkey     2024 non-null   object  
 2   shapeName   2024 non-null   object  
 3   shapeISO    2024 non-null   object  
 4   shapeID     2024 non-null   object  
 5   shapeGroup  2024 non-null   object  
 6   shapeType   2024 non-null   object  
 7   pop_count   2024 non-null   float64 
dtypes: float64(1), geometry(1), object(6)
memory usage: 142.3+ KB


In [7]:
admin_grids_gdf.head(2)

,geometry,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,pop_count
0,"POLYGON ((124.03564 -9.34067, 124.03564 -9.318...",31011220203121,Nitibe,None,TLS-ADM2-3_0_0-B58,TLS,ADM2,102.251936
1,"POLYGON ((124.03564 -9.36235, 124.03564 -9.340...",31011220203123,Nitibe,None,TLS-ADM2-3_0_0-B58,TLS,ADM2,992.492772


In [ ]:
# Uncomment to view the generated grid on an interactive map
# admin_grids_gdf.explore()

## Save to file

In [9]:
rollout_date = "-".join(os.getcwd().split("/")[-2].split("-")[:3])
grid_save_path = f"./{rollout_date}-{COUNTRY_CODE}-rollout-grids.geojson"
admin_grids_gdf.to_file(grid_save_path, driver="GeoJSON", index=False)